In [1]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as ma
import model_circular_economy as mc
import model_ippu as mi
import model_socioeconomic as ms

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(ms)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:720: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:720: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_socioeconomic' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_socioeconomic.py'>

In [545]:
area_crops = np.array(df_afolu_trajectories["area_gnrl_country_ha"])*np.array(df_afolu_trajectories["frac_lndu_initial_croplands"])
#print(np.round(area_crops[0]))
aa = area_crops[0]*float(df_afolu_trajectories[[x for x in df_afolu_trajectories.columns if ("frac_agrc_initial_area" in x) and ("sugar" in x)]].iloc[0])
int(np.round(aa))
 

20522288

In [3]:
df_project_afolu[[x for x in df_project_afolu.columns if "harvested_" in x]].head()

,emission_co2e_co2_frst_harvested_wood_products
0,-0.406721
1,-0.462721
2,-0.537133
3,-0.596382
4,-0.641623


In [21]:
varlist = []
for subsec in sa.model_attributes.get_sector_subsectors("AFOLU"):
    varlist += sa.model_attributes.build_varlist(subsec, variable_type = "output")
varlist = [x for x in varlist if "emission_co2e" in x]
varlist.sort()

for k in varlist:
    print(k)

emission_co2e_ch4_agrc_anaerobicdom_rice
emission_co2e_ch4_frst_methane_mangroves
emission_co2e_ch4_frst_methane_primary
emission_co2e_ch4_frst_methane_secondary
emission_co2e_ch4_lndu_wetlands
emission_co2e_ch4_lsmm_anaerobic_digester
emission_co2e_ch4_lsmm_anaerobic_lagoon
emission_co2e_ch4_lsmm_composting
emission_co2e_ch4_lsmm_daily_spread
emission_co2e_ch4_lsmm_deep_bedding
emission_co2e_ch4_lsmm_dry_lot
emission_co2e_ch4_lsmm_incineration
emission_co2e_ch4_lsmm_liquid_slurry
emission_co2e_ch4_lsmm_paddock_pasture_range
emission_co2e_ch4_lsmm_poultry_manure
emission_co2e_ch4_lsmm_storage_solid
emission_co2e_ch4_lvst_entferm_buffalo
emission_co2e_ch4_lvst_entferm_cattle_dairy
emission_co2e_ch4_lvst_entferm_cattle_nondairy
emission_co2e_ch4_lvst_entferm_chickens
emission_co2e_ch4_lvst_entferm_goats
emission_co2e_ch4_lvst_entferm_horses
emission_co2e_ch4_lvst_entferm_mules
emission_co2e_ch4_lvst_entferm_pigs
emission_co2e_ch4_lvst_entferm_sheep
emission_co2e_co2_agrc_biomass_bevs_and

In [4]:
# df_project_afolu[[x for x in df_project_afolu.columns if "emission_co2e_subsector_total" in x]]
importlib.reload(mi)
#importlib.reload(me)
importlib.reload(ma)
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
# test
#df_afolu_trajectories = pd.read_csv("/Users/jsyme/Downloads/data_complete_future_2022_07_19.csv")
#df_afolu_trajectories["frac_agrc_agriculture_production_lost"] = 0.21 # temp
#df_afolu_trajectories = df_afolu_trajectories[df_afolu_trajectories["Nation"] == "brazil"].reset_index(drop = True)
#df_afolu_trajectories["ef_soil_c_cultivated_organic_tropical_crop_grass_tonne_per_ha"] = 20
#df_afolu_trajectories["ef_soil_c_cultivated_organic_temperate_crop_grass_tonne_per_ha"] = 7.5

model_afolu = ma.AFOLU(sa.model_attributes)
model_circ = mc.CircularEconomy(sa.model_attributes)
model_ippu = mi.IPPU(sa.model_attributes)
warnings.filterwarnings("ignore")
t0 = time.time()#
#df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 70
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_wood"] = [0.1, 0.2, 0.3, 0.4, 0.9, 1.2] + [1.2 for x in range(30)]
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_cement"] = np.array([1, 0.9, 0.8, 0.7] + [0 for x in range(32)]) 
#df_afolu_trajectories["prodinit_ippu_cement_tonne"] = 10
#df_afolu_trajectories["frac_scoe_heat_energy_residential_solid_biomass"] = [0.13, 0.12,0.11,0.1, 0.09, 0.05, 0.01] + [0.01 for x in range(29) ]
df_project_afolu = model_afolu.project(df_afolu_trajectories)
df_project_ce = model_circ.project(df_afolu_trajectories)
t1 = time.time()
#print(t1 - t0)
#df_project_circ = model_circ.project(df_afolu_trajectories)
#df_ippu_in = sf.merge_output_df_list([df_afolu_trajectories, df_project_circ], sa.model_attributes, "concatenate")

#df_project_ippu = model_ippu.project(df_ippu_in)
#sa.model_attributes.get_standard_variables(df_project_ippu, model_ippu.modvar_ippu_qty_total_production)
#df_project_afolu[[x for x in df_project_afolu if "emission_co2e_n2o_soil" in x]]#/.sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu if "frst" in x]]

#print(df_project_afolu["emission_co2e_co2_frst_harvested_wood_products"])
df_project_afolu[[x for x in df_project_afolu if "qty_agrc" in x and "lost" in x]]
df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("soil" in x)]]
df_project_afolu[[x for x in df_project_afolu if ("emission" in x) and ("agrc" in x)]]

,emission_co2e_ch4_agrc_anaerobicdom_rice,emission_co2e_co2_agrc_biomass_bevs_and_spices,emission_co2e_co2_agrc_biomass_fruits,emission_co2e_co2_agrc_biomass_nuts,emission_co2e_co2_agrc_biomass_other_woody_perennial,emission_co2e_n2o_agrc_biomass_burning,emission_co2e_n2o_agrc_crop_residues,emission_co2e_co2_agrc_soil_carbon_mineral_soils,emission_co2e_co2_agrc_soil_carbon_organic_soils,emission_co2e_subsector_total_agrc
0,0.084367,0.012101,0.010168,0.007161,0.012586,0.047764,0.044994,0.450273,0.994542,1.663954
1,0.085395,0.012297,0.010292,0.007248,0.012646,0.048791,0.046005,0.450273,0.996525,1.669472
2,0.086926,0.012577,0.010476,0.007378,0.012752,0.049780,0.046962,0.023754,1.013216,1.263823
3,0.088545,0.012862,0.010671,0.007515,0.012875,0.050710,0.047850,0.164903,1.031224,1.427157
4,0.090271,0.013166,0.010879,0.007662,0.013003,0.051749,0.048836,0.323432,1.050839,1.609836
5,0.092032,0.013478,0.011092,0.007811,0.013131,0.052835,0.049858,0.460171,1.071306,1.771715
6,0.093867,0.013808,0.011313,0.007967,0.013260,0.054030,0.050976,0.544206,1.093097,1.882523
7,0.095774,0.014156,0.011543,0.008129,0.013388,0.055325,0.052179,0.605691,1.116239,1.972422
8,0.097781,0.014532,0.011785,0.008299,0.013515,0.056755,0.053502,0.646110,1.141142,2.043421
9,0.099797,0.014922,0.012028,0.008471,0.013627,0.058335,0.054956,0.668746,1.166825,2.097708


In [290]:

df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 10
df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_wood"] = [0.1, 0.2, 0.3, 0.4, 0.9, 1.2] + [1.2 for x in range(30)]
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_cement"] = np.array([1, 0.9, 0.8, 0.7] + [0 for x in range(32)]) 
#df_afolu_trajectories["prodinit_ippu_cement_tonne"] = 10
#df_afolu_trajectories["frac_scoe_heat_energy_residential_solid_biomass"] = [0.13, 0.12,0.11,0.1, 0.09, 0.05, 0.01] + [0.01 for x in range(29) ]
df_project_afolu = model_afolu.project(df_afolu_trajectories)

In [361]:
#sa.model_attributes.build_varlist("IPPU")#, model_ippu.modvar_ippu_max_recycled_material_ratio)
#df_afolu_trajectories.shape


In [375]:
model_ippu = mi.IPPU(sa.model_attributes)
varm = model_ippu.modvar_ippu_max_recycled_material_ratio

arr1 = sa.model_attributes.get_standard_variables(df_afolu_trajectories, varm, return_type = "array_base", expand_to_all_cats = True)
arr1[:, attr_ippu.get_key_value_index("paper")]
#sa.model_attributes.get_scalar("Initial Non-Energy Fuel Use", "mass")
#sa.model_attributes.get_variable_characteristic("Initial Non-Energy Fuel Use", "$UNIT-MASS$")

array([0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6,
       0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6,
       0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])

In [377]:
importlib.reload(ds)
importlib.reload(ma)

model_afolu = ma.AFOLU(sa.model_attributes)
#model_circ = mc.CircularEconomy(sa.model_attributes)
#model_ippu = mi.IPPU(sa.model_attributes)
warnings.filterwarnings("ignore")
df_project_afolu = model_afolu.project(df_afolu_trajectories)

df_project_afolu[[x for x in df_project_afolu if "emission_co2e_n2o_soil" in x]].sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu if "frst" in x and "emission" in x]]#.sum(axis = 1)

[3.58662904 3.64319854 3.70066026 3.7590283  3.81831693 3.87854068
 3.93971431 4.00185278 4.06497132 4.12908539 4.19421069 4.27868884
 4.35010397 4.40912514 4.4730651  4.53550754 4.60186845 4.67053456
 4.74379887 4.8214971  4.90001911 4.97937974 5.05958249 5.14063644
 5.22254875 5.30532983 5.388989   5.4735475  5.55901905 5.64543633
 5.73281802 5.82118139 5.91054204 6.00091228 6.09231211 6.18475895
 6.27826589 6.37285301 6.46852925 6.56529557 6.66316441 6.76215277
 6.8622802  6.96356842 7.06603641 7.16970685]


,emission_co2e_co2_frst_sequestration_mangroves,emission_co2e_co2_frst_sequestration_primary,emission_co2e_co2_frst_sequestration_secondary,emission_co2e_ch4_frst_methane_mangroves,emission_co2e_ch4_frst_methane_primary,emission_co2e_ch4_frst_methane_secondary,emission_co2e_co2_frst_forest_fires,emission_co2e_co2_frst_harvested_wood_products,emission_co2e_subsector_total_frst
0,-0.098780,-5.237938,-1.105878,0.001312,0.060891,0.025712,0.000180,-0.406721,-6.761223
1,-0.098629,-5.223194,-1.093686,0.001310,0.060720,0.025428,0.000178,-0.462721,-6.790593
2,-0.098375,-5.208907,-1.078163,0.001307,0.060554,0.025067,0.000176,-0.537133,-6.835473
3,-0.098081,-5.195096,-1.061796,0.001303,0.060393,0.024687,0.000174,-0.596382,-6.864798
4,-0.097764,-5.181750,-1.045307,0.001299,0.060238,0.024303,0.000172,-0.641623,-6.880432
5,-0.097441,-5.168867,-1.029302,0.001295,0.060088,0.023931,0.000170,-0.690918,-6.901045
6,-0.097092,-5.156436,-1.012820,0.001290,0.059944,0.023548,0.000167,-0.737577,-6.918976
7,-0.096713,-5.144451,-0.995659,0.001285,0.059804,0.023149,0.000165,-0.787183,-6.939603
8,-0.096293,-5.132905,-0.977297,0.001279,0.059670,0.022722,0.000162,-0.837997,-6.960659
9,-0.095844,-5.121791,-0.958192,0.001273,0.059541,0.022278,0.000160,-0.892318,-6.984893


In [ ]:
df

In [65]:
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_afolu_trajectories.shape                      
#self.model_attributes.get_standard_variables(df_afolu_trajectories, self.modvar_lvst_animal_weight, True, "array_base", expand_to_all_cats = True)                        

(36, 1578)

In [15]:
#vars_afolu = sa.model_attributes.build_varlist("Soil Management", variable_type = "input")
vars_afolu = sa.model_attributes.build_varlist("Fugitive Emissions", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Forest", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Land Use", variable_type = "input")
vars_afolu = [x for x in vars_afolu if (x not in df_afolu_trajectories.columns)]
vars_afolu.sort()

for k in vars_afolu:
    print(k)
    

ef_fgtv_ch4_futgitive_emissions_fuel_coal
ef_fgtv_ch4_futgitive_emissions_fuel_natural_gas
ef_fgtv_ch4_futgitive_emissions_fuel_oil
ef_fgtv_ch4_futgitive_emissions_with_full_capture_fuel_oil
ef_fgtv_co2_futgitive_emissions_fuel_coal
ef_fgtv_co2_futgitive_emissions_fuel_natural_gas
ef_fgtv_co2_futgitive_emissions_fuel_oil
ef_fgtv_n2o_futgitive_emissions_fuel_coal
ef_fgtv_n2o_futgitive_emissions_fuel_natural_gas
ef_fgtv_n2o_futgitive_emissions_fuel_oil
exportsinit_fgtv_fuel_coal_tonne
exportsinit_fgtv_fuel_natural_gas_tonne
exportsinit_fgtv_fuel_oil_tonne
frac_fgtv_fugitive_capturable_ch4_captured_fuel_coal
frac_fgtv_fugitive_capturable_ch4_captured_fuel_natural_gas
frac_fgtv_fugitive_capturable_ch4_captured_fuel_oil
frac_fgtv_fugitive_ch4_flared_fuel_coal
importsinit_fgtv_fuel_coal_tonne
importsinit_fgtv_fuel_natural_gas_tonne
importsinit_fgtv_fuel_oil_tonne
prodinit_fgtv_fuel_coal_tonne
prodinit_fgtv_fuel_natural_gas_tonne
prodinit_fgtv_fuel_oil_tonne
ratio_fgtv_fuel_imports_to_exports

In [409]:
print(model_afolu.modvar_lsmm_emissions_n2o)

:math:\text{N}_2\text{O} Emissions from Manure Management


In [80]:

#df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")





In [16]:
dict_arrs_lsmm_frac_manure = sa.model_attributes.get_multivariables_with_bounded_sum_by_category(
    df_afolu_trajectories,
    model_afolu.modvar_lvst_list_mm_fractions,
    1,
    force_sum_equality = True,
    msg_append = "Energy fractions by category do not sum to 1. See definition of dict_arrs_inen_frac_energy."
)

In [26]:
sa.model_attributes.get_standard_variables(df_afolu_trajectories, model_afolu.modvar_lvst_genfactor_volatile_solids)

,genfactor_lvst_daily_volatile_solids_buffalo,genfactor_lvst_daily_volatile_solids_cattle_dairy,genfactor_lvst_daily_volatile_solids_cattle_nondairy,genfactor_lvst_daily_volatile_solids_chickens,genfactor_lvst_daily_volatile_solids_goats,genfactor_lvst_daily_volatile_solids_horses,genfactor_lvst_daily_volatile_solids_mules,genfactor_lvst_daily_volatile_solids_pigs,genfactor_lvst_daily_volatile_solids_sheep
0,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
1,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
2,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
3,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
4,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
5,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
6,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
7,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
8,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
9,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083


In [412]:
df_emissions = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Emissions_Totals_E_All_Data_(Normalized)/Emissions_Totals_E_All_Data_(Normalized).csv")



In [424]:
ai = set(df_emissions["Item"])
ai - all_items

{'Drained organic soils',
 'Drained organic soils (CO2)',
 'Drained organic soils (N2O)'}

In [517]:
all_items = set(df_emissions[
    df_emissions["Year"].isin([2019]) &
    df_emissions["Area"].isin(["Mexico"])
]["Item"])

all_elems = set([x for x in list(df_emissions["Element"]) if "Emissions (CO2eq) (AR5)" in x])

df_emissions[
    df_emissions["Year"].isin([2019]) &
    df_emissions["Area"].isin(["Mexico"]) & 
    df_emissions["Element"].isin(all_elems)
].to_csv("/Users/jsyme/Desktop/calc_fao_mb.csv", index = None, encoding = "UTF-8")

In [419]:
df_tmp = pd.read_csv("/Users/jsyme/Downloads/FAOSTAT_data_7-19-2022 (1).csv")


In [432]:
#df_tmp
all_elems

{'Emissions (CO2eq) (AR5)',
 'Emissions (CO2eq) from CH4 (AR5)',
 'Emissions (CO2eq) from N2O (AR5)'}

In [430]:
list(df_emissions["Element"])

['Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
 'Emissions (CH4)',
